In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings

# Supprimer les warnings pour une meilleure lisibilité
warnings.filterwarnings("ignore")

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# On suppose que la granularité horaire est pertinente
# target_series contient déjà les colonnes ['date_heure', 'nb_interventions', 'nb_appels']

# Préparation des features temporelles
target_series['hour'] = target_series['date_heure'].dt.hour
target_series['dayofweek'] = target_series['date_heure'].dt.dayofweek
target_series['month'] = target_series['date_heure'].dt.month
target_series['year'] = target_series['date_heure'].dt.year

# Décalage pour prédire la prochaine heure (ex: t+1)
target_series['nb_appels'] = target_series['nb_appels'].shift(-1)

# On retire la dernière ligne qui n'a pas de cible
target_series = target_series.dropna(subset=['nb_appels_next'])

X = target_series[['hour', 'dayofweek', 'month', 'year', 'nb_appels']]
y = target_series['nb_appels_next']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Modèle simple
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédiction sur le test
y_pred = model.predict(X_test)

# Affichage rapide des résultats
plt.figure(figsize=(15,5))
plt.plot(y_test.values, label='Réel')
plt.plot(y_pred, label='Prédit')
plt.title("Prédiction du nombre d'appels (prochaine heure)")
plt.legend()
plt.show()

NameError: name 'target_series' is not defined

In [ ]:
# On va prédire le nombre d'interventions (nb_interventions) à l'heure suivante

# Création de la colonne cible pour la prochaine heure
target_series['nb_interventions_next'] = target_series['nb_interventions'].shift(-1)

# On retire la dernière ligne qui n'a pas de cible
target_series = target_series.dropna(subset=['nb_interventions_next'])

X = target_series[['hour', 'dayofweek', 'month', 'year', 'nb_interventions']]
y = target_series['nb_interventions_next']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Modèle simple
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédiction sur le test
y_pred = model.predict(X_test)

# Affichage rapide des résultats
plt.figure(figsize=(15,5))
plt.plot(y_test.values, label='Réel')
plt.plot(y_pred, label='Prédit')
plt.title("Prédiction du nombre d'interventions (prochaine heure)")
plt.legend()
plt.show()

In [ ]:
# Ajout de la durée totale (temps d'intervention) dans les features pour l'exploration
target_series['duree_totale_next'] = target_series['duree_totale'].shift(-1)

# On retire la dernière ligne qui n'a pas de cible
target_series = target_series.dropna(subset=['duree_totale_next'])

X = target_series[['hour', 'dayofweek', 'month', 'year', 'nb_interventions', 'duree_totale']]
y = target_series['duree_totale_next']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Modèle simple
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédiction sur le test
y_pred = model.predict(X_test)

# Affichage rapide des résultats
plt.figure(figsize=(15,5))
plt.plot(y_test.values, label='Réel')
plt.plot(y_pred, label='Prédit')
plt.title("Prédiction de la durée totale d'intervention (prochaine heure)")
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# --- SARIMA pour nb_interventions (déjà fait ci-dessus) ---
# On conserve les prédictions SARIMA
sarima_pred_inter = pred_inter

# --- ARIMA pour nb_interventions ---
arima_model_inter = ARIMA(interventions_series, order=(1,1,1))
arima_results_inter = arima_model_inter.fit()
arima_pred_inter = arima_results_inter.forecast(steps=n_test)

# --- Comparaison des modèles pour nb_interventions ---
mse_sarima_inter = mean_squared_error(interventions_series[-n_test:], sarima_pred_inter)
mse_arima_inter = mean_squared_error(interventions_series[-n_test:], arima_pred_inter)

plt.figure(figsize=(15,5))
plt.plot(interventions_series[-n_test:], label='Réel')
plt.plot(sarima_pred_inter, label='SARIMA')
plt.plot(arima_pred_inter, label='ARIMA')
plt.title("Comparaison SARIMA vs ARIMA - nb_interventions")
plt.legend()
plt.show()

print(f"MSE SARIMA (nb_interventions): {mse_sarima_inter:.2f}")
print(f"MSE ARIMA (nb_interventions): {mse_arima_inter:.2f}")
if mse_sarima_inter < mse_arima_inter:
    print("SARIMA est meilleur pour nb_interventions.")
else:
    print("ARIMA est meilleur pour nb_interventions.")

# --- SARIMA pour duree_totale (déjà fait ci-dessus) ---
sarima_pred_duree = pred_duree

# --- ARIMA pour duree_totale ---
arima_model_duree = ARIMA(duree_series, order=(1,1,1))
arima_results_duree = arima_model_duree.fit()
arima_pred_duree = arima_results_duree.forecast(steps=n_test)

# --- Comparaison des modèles pour duree_totale ---
mse_sarima_duree = mean_squared_error(duree_series[-n_test:], sarima_pred_duree)
mse_arima_duree = mean_squared_error(duree_series[-n_test:], arima_pred_duree)

plt.figure(figsize=(15,5))
plt.plot(duree_series[-n_test:], label='Réel')
plt.plot(sarima_pred_duree, label='SARIMA')
plt.plot(arima_pred_duree, label='ARIMA')
plt.title("Comparaison SARIMA vs ARIMA - duree_totale")
plt.legend()
plt.show()

print(f"MSE SARIMA (duree_totale): {mse_sarima_duree:.2f}")
print(f"MSE ARIMA (duree_totale): {mse_arima_duree:.2f}")
if mse_sarima_duree < mse_arima_duree:
    print("SARIMA est meilleur pour duree_totale.")
else:
    print("ARIMA est meilleur pour duree_totale.")